<a href="https://colab.research.google.com/github/Saadha123/Intellihack_MorattuPasanga_2/blob/main/Intent_Classification_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install tensorflow==2.7.0


ERROR: Could not find a version that satisfies the requirement tensorflow==2.7.0 (from versions: 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0, 2.11.1, 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0, 2.14.1, 2.15.0rc0, 2.15.0rc1, 2.15.0, 2.15.0.post1, 2.15.1, 2.16.0rc0, 2.16.1)
ERROR: No matching distribution found for tensorflow==2.7.0


In [ ]:
# Import Data Manipulation Libraries
import pandas as pd
import numpy as np
# Import Data visualization Libraries
import matplotlib.pyplot as plt
import seaborn as sns
#To import different metrics
from sklearn import metrics
from sklearn.metrics import (
    f1_score,
    accuracy_score,
    recall_score,
    precision_score,
    confusion_matrix
)

# Import JSON Library
import json

# Import TensorFlow
import tensorflow as tf
# Import Transformers
from transformers import BertTokenizer, TFBertForSequenceClassification

#Importing classback API
from keras import callbacks
from tensorflow.keras import backend
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD


# Library to avoid the warnings
import warnings
warnings.filterwarnings("ignore")
# To supress scientific notations for a dataframe
pd.set_option("display.float_format", lambda x: "%.2f" % x)

In [ ]:
# Import drive to load training data file
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
# Path to train data file
train_data_file_path = '/content/drive/My Drive/intellihack/intent_classification/is_train.json'

In [ ]:
with open(train_data_file_path, 'r') as file:
    json_data = json.load(file)

In [ ]:
# Convert the JSON train data to a pandas DataFrame
train_data = pd.DataFrame(json_data, columns=['text', 'intent'])

In [ ]:
# View the first 5 rows of the dataset
train_data.head()

,text,intent
0,what expression would i use to say i love you ...,translate
1,can you tell me how to say 'i do not speak muc...,translate
2,"what is the equivalent of, 'life is good' in f...",translate
3,"tell me how to say, 'it is a beautiful morning...",translate
4,"if i were mongolian, how would i say that i am...",translate


In [ ]:
# View the last 5 rows of the dataset
train_data.tail()

,text,intent
14995,can you explain why my card was declined,card_declined
14996,how come starbucks declined my card when i tri...,card_declined
14997,how come my card was not accepted yesterday,card_declined
14998,find out what happened to make my card get dec...,card_declined
14999,why was my card declined at safeway,card_declined


In [ ]:
# View the shape of the dataset
train_data.shape


(15000, 2)

In [ ]:
# View the data types of the columns
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    15000 non-null  object
 1   intent  15000 non-null  object
dtypes: object(2)
memory usage: 234.5+ KB


In [ ]:
# Checking for unique values
train_data.nunique()

text      15000
intent      150
dtype: int64

In [ ]:
# Checking for Duplicate Entries
train_data.duplicated().sum()

0

In [ ]:
# Checking for Missing Values
train_data.isna().sum()

text      0
intent    0
dtype: int64

# **Preprocess data**

In [ ]:
# Extract intent labels and text data
# used as the input variables for the model
train_text = train_data['text']

# used as the target variable for predicting the intent
train_label = train_data['intent']

In [ ]:
# Cconvert categorical labels (intent labels, in this case) into numerical values
unique_intents = train_label.unique()
intent_to_label = {intent: i for i, intent in enumerate(unique_intents)}

# Inverse the intent_to_label mapping to create a label_to_intent mapping (to be used later on for prediction)
label_to_intent = {label: intent for intent, label in intent_to_label.items()}

# The target variable = labels in numberical values
train_label_y = [intent_to_label[intent] for intent in train_label]

# Convert labels (intent numberical labels) to TensorFlow tensors
train_label_y = tf.convert_to_tensor(train_label_y)

In [ ]:
# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the training texts
train_encodings = tokenizer(train_text.tolist(), truncation=True, padding=True, max_length=128, return_tensors='tf')

# Prepare Input Data for Bert Model
train_input_data = {
    'input_ids': train_encodings['input_ids'],
    'attention_mask': train_encodings['attention_mask']
}

In [ ]:
# Path to train data file
valid_data_file_path = '/content/drive/MyDrive/intellihack/intent_classification/is_val.json'


In [ ]:
# Open and read the JSON file
with open(valid_data_file_path, 'r') as file:
    json_data = json.load(file)

In [ ]:
# Convert the JSON data to a pandas DataFrame
valid_data = pd.DataFrame(json_data, columns=['text', 'intent'])

In [ ]:
# Extract intent labels and text data
valid_label = valid_data['intent']
valid_text = valid_data['text']

In [ ]:
# Map intents to numerical labels
# convert categorical labels (intent labels, in this case) into numerical values
unique_intents = valid_label.unique()
intent_to_label = {intent: i for i, intent in enumerate(unique_intents)}
valid_label_y = [intent_to_label[intent] for intent in valid_label]
print('There are',len(intent_to_label),'labels')


There are 150 labels


In [ ]:
# Convert labels (intent numberical labels) to TensorFlow tensors
valid_label_y = tf.convert_to_tensor(valid_label_y)

In [ ]:
# Tokenize the training texts
valid_encodings = tokenizer(valid_text.tolist(), truncation=True, padding=True, max_length=128, return_tensors='tf')


In [ ]:

# input data for the model
valid_input_data = {
    'input_ids': valid_encodings['input_ids'],
    'attention_mask': valid_encodings['attention_mask']
}

In [ ]:
# Load the pre-trained BERT model for sequence classification
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(intent_to_label))


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# # using Adams optimized with learning rate 1e-5
# optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001)


In [ ]:
optimizer = SGD(learning_rate=0.01, momentum=0.9)

In [ ]:
# as opposed to one-hot encoded arrays.
# from_logits=True, the SparseCategoricalCrossentropy loss function internally applies a softmax function to the logits before computing the actual loss value.
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
# Compiling the model with the optimizer, loss func and metrics
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])


ValueError: Could not interpret optimizer identifier: <keras.src.optimizers.sgd.SGD object at 0x7ac71ceadcc0>

In [ ]:
# Obtain the summary of the model
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_75 (Dropout)        multiple                  0 (unused)
                                                                 
 classifier (Dense)          multiple                  115350    
                                                                 
Total params: 109597590 (418.08 MB)
Trainable params: 109597590 (418.08 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
es_cb = callbacks.EarlyStopping(monitor='val_loss', min_delta=0.01, patience=5)


In [ ]:
# Train the model
history = model.fit(
    train_input_data,
    train_label_y,
    validation_data=(valid_input_data,valid_label_y),
    epochs=7,
    batch_size=32,
    callbacks=[es_cb]
)

RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.

In [ ]:
# Capturing learning history per epoch and displaying it
hist  = pd.DataFrame(history.history)
hist['epoch'] = history.epoch

# Plotting accuracy at different epochs
plt.plot(hist['loss'])
plt.plot(hist['val_loss'])
plt.legend(("tin" , "valid") , loc =0)

In [ ]:
# 'history' is the result from model.fit()
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
epochs = range(1, len(acc) + 1)

# Plotting training and validation accuracy
plt.figure(figsize=(8, 5))
plt.plot(epochs, acc,  label='Training Accuracy')
plt.plot(epochs, val_acc,  label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# Path to train data file
test_data_file_path = '//content/drive/MyDrive/intellihack/intent_classification/is_val.json'


In [ ]:
# Open and read the val_datasetJSON file
with open(test_data_file_path, 'r') as file:
    json_data = json.load(file)

# Convert the JSON data to a pandas DataFrame
test_data = pd.DataFrame(json_data, columns=['text', 'intent'])

# Extract intent labels and text data
test_label = test_data['intent']
test_text = test_data['text']

# Map intents to numerical labels
# convert categorical labels (intent labels, in this case) into numerical values
unique_intents = test_data['intent'].unique()
intent_to_label = {intent: i for i, intent in enumerate(unique_intents)}
test_label_y = [intent_to_label[intent] for intent in test_label]

# Convert labels (intent numberical labels) to TensorFlow tensors
test_label_y = tf.convert_to_tensor(test_label_y)

# Tokenize the training texts
test_encodings = tokenizer(test_text.tolist(), truncation=True, padding=True, max_length=128, return_tensors='tf')

# Prepare the input data for the model
test_input_data = {
    'input_ids': test_encodings['input_ids'],
    'attention_mask': test_encodings['attention_mask']
}

In [ ]:
# Generate predictions
test_predictions = model.predict(test_input_data)

# `test_predictions` is a TFSequenceClassifierOutput object
logits = test_predictions.logits

# Convert logits to probabilities (optional, for better interpretability)
probabilities = tf.nn.softmax(logits, axis=-1)

# Convert probabilities to class labels
test_pred_labels = tf.argmax(probabilities, axis=-1)

94/94 [==============================] - 651s 7s/step


In [ ]:
# To supress scientific notations for a dataframe
pd.set_option("display.float_format", lambda x: "%.3f" % x)

def model_performance_classification(target,pred):
    # Calculate accuracy
    accuracy = accuracy_score(target, pred)

    # Calculate precision
    precision = precision_score(target, pred, average='weighted', zero_division=0)

    # Calculate recall
    recall = recall_score(target, pred, average='weighted', zero_division=0)

    # Calculate F1 score
    f1 = f1_score(target, pred, average='weighted', zero_division=0)

    # creating a dataframe of metrics
    df_perf = pd.DataFrame(
        {
            "Accuracy": accuracy,
            "Recall": recall,
            "Precision": precision,
            "F1": f1

        },
        index=[0],
    )

    return df_perf


model_performance = model_performance_classification(test_label_y,test_pred_labels)

model_performance

,Accuracy,Recall,Precision,F1
0,0.007,0.007,0.000,0.000


In [ ]:
input_text = ['I would need 5 days for my vacation']

# Tokenize the input text text
input_text_encodings = tokenizer(input_text, truncation=True, padding=True, max_length=128, return_tensors='tf')

input_text_input_data = {
    'input_ids': input_text_encodings['input_ids'],
    'attention_mask': input_text_encodings['attention_mask']
}

# Generate predictions
input_text_predictions = model.predict(input_text_input_data)

# input_text_predictions is a TFSequenceClassifierOutput object
logits = input_text_predictions.logits

# Convert logits to probabilities (optional, for better interpretability)
probabilities = tf.nn.softmax(logits, axis=-1)

# Convert probabilities to class labels
input_text_pred_labels = tf.argmax(probabilities, axis=-1)

# Inverse the intent_to_label mapping to create a label_to_intent mapping
# label_to_intent = {label: intent for intent, label in intent_to_label.items()}

# Convert the predicted label indices to numpy if they aren't already
input_text_pred_labels_np = input_text_pred_labels.numpy()

# Map the numerical label to its corresponding intent using the inverse mapping
predicted_intents = [label_to_intent[label] for label in input_text_pred_labels_np]

print("Predicted intents:", predicted_intents)

1/1 [==============================] - 26s 26s/step
Predicted intents: ['current_location']


In [ ]:
    predicted_intent_idx = np.argmax(prediction)

    # Get the confidence score for the predicted intent
    confidence_score = prediction[0][predicted_intent_idx]

NameError: name 'prediction' is not defined

In [ ]:
    # Check if the confidence score meets the threshold
    threshold = 0.7
    if confidence_score >= threshold:
        # Return the predicted intent and confidence score
        intent = list(dataset["intents"].keys())[predicted_intent_idx]
        return intent, confidence_score
    else:
        # Return the fallback response
        return " Intent could not be confidently determined", None